### Download structure data from Cybase

In [204]:
##import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import urllib
from urllib.request import urlopen
import os
import re

In [5]:
##get cybase id
data = pd.read_excel('ID.xlsx', header=0)
print(len(data))
data.head()

1257


,CPID,Cybase_id
0,CP00140,720
1,CP00164,654
2,CP00233,698
3,CP00295,518
4,CP00353,521


In [26]:
cbID = []
for i in range(len(data)):
    ids = str(data.Cybase_id[i]).replace(" ", "").split("##")
    for cid in ids:
        cbID.append(cid)
print(len(cbID))        
cbID = list(set(cbID))
print(len(cbID))    

1258
1236


In [28]:
##download cybase entry info
for cbid in cbID:
    if not os.path.exists("htmls/"+cbid+".html"):
        response = urlopen('https://www.cybase.org.au/index.php?page=card&table=protein&id='+cbid)

        if response.code == 200:
            with open("htmls/"+cbid+".html", "wb") as file:
                file.write(response.read())
        else:
            print("请求失败！状态码：%d" % response.status_code)

In [84]:
# get general information table
def transpose_general_info(table_data, ID):
    general_info = pd.read_html(table_data)
    general_info = general_info[0].transpose()
    new_header = general_info.iloc[0]
    general_info = general_info[1:]
    general_info.columns = new_header
    general_info['ID'] = ID
    return(general_info)

GeneralInfo = pd.DataFrame()
for cbid in cbID:
    data_entry = BeautifulSoup(open("htmls/"+cbid+".html"), 'lxml')
    tables = data_entry.find_all('table')
    general_info = transpose_general_info(tables[0].prettify(), cbid)
    GeneralInfo = pd.concat([GeneralInfo, general_info], axis=0, ignore_index=True)

In [86]:
GeneralInfo.head(n=10)
GeneralInfo.to_excel('Cybase_General_Info.xlsx', header=True, index=False)

In [226]:
# get assay information
def get_assay_info(data, ID):
    ASSAY = pd.DataFrame()
    for child in data.children:
        assay_type = child.td.text
        assay_info = pd.read_html(child.find('table').prettify())[0]
        
        #assay_id& pmid
        uids = []
        assay_ids = []
        trs = child.table.find_all('tr')
        for tr in trs:
            tds = tr.find_all('td')
            if tds[1].a is not None:
                pattern = re.compile(r'list_uids=[0-9]+')
                uid = pattern.findall(tds[1].a.attrs['href'])
                uid = re.compile(r'[0-9]+').findall(uid[0])
                uids.append(uid[0])
            else:
                uids.append(np.NAN)
                
            if tds[0].a is not None:
                assay_id = re.compile(r'[0-9]+').findall(tds[0].a.attrs['href'])
                assay_ids.append(assay_id[0])
            else:
                assay_ids.append(np.NAN)
        
        assay_info['cybase_assay_ids'] = assay_ids
        assay_info['assay_ref_uids'] = uids
        assay_info['assay_type'] = assay_type
        ASSAY = pd.concat([ASSAY, assay_info], axis=0, ignore_index=True)
    ASSAY['ID'] = ID
    return(ASSAY)

ASSAYs = pd.DataFrame()
for cbid in cbID:
    data_entry = BeautifulSoup(open("htmls/"+cbid+".html"), 'lxml')
    tables = data_entry.find_all('table', attrs={'class':'cardtable'})

    assays = tables[1].find('table')
    if assays is not None:
        ASSAY = get_assay_info(assays, cbid)
    else:
        ASSAY = pd.DataFrame([[np.NAN, np.NAN, np.NAN, np.NAN, tables[1].find('i').text, cbid]], 
                             columns=[0,1,'cybase_assay_ids','assay_ref_uids','assay_type','ID'])
    ASSAYs = pd.concat([ASSAYs, ASSAY], axis=0, ignore_index=True)
ASSAYs.columns = ['assay_summary','assay_ref','cybase_assay_ids','assay_ref_uids','assay_type','ID']
ASSAYs.head(10)

,assay_summary,assay_ref,cybase_assay_ids,assay_ref_uids,assay_type,ID
0,NaN,NaN,NaN,NaN,No assays found,992
1,NaN,NaN,NaN,NaN,No assays found,1610
2,NaN,NaN,NaN,NaN,No assays found,913
3,NaN,NaN,NaN,NaN,No assays found,25
4,NaN,NaN,NaN,NaN,No assays found,1345
5,NaN,NaN,NaN,NaN,No assays found,1574
6,NaN,NaN,NaN,NaN,No assays found,1292
7,NaN,NaN,NaN,NaN,No assays found,656
8,NaN,NaN,NaN,NaN,No assays found,261
9,RTD-1-3 shown to bind with equal affinity to E...,Tran D et al. (2002) J Biol Chem 277 :307...,27,11675394,Membrane Binding Assay,89


In [227]:
ASSAYs.to_excel('Cybase_Assays_Info.xlsx', header=True, index=False)

In [331]:
def structure_filter(tag):
    return tag.name == 'td' and tag.a and tag.previous_sibling.previous_sibling.text == 'Structure'

def get_structure_info(table_data, ID):
    struc_td = table_data.find_all(structure_filter)
    Struc_ids = []
    Struc_names = []
    if len(struc_td) > 0:
        struc_a = struc_td[0].find_all('a')
        for a in struc_a:
            struc_id = re.compile(r'[0-9]+').findall(a['href'])[0]
            struc_name = a.text
            Struc_ids.append(struc_id)
            Struc_names.append(struc_name)
        #print(Struc_ids)
        Struc_info = pd.DataFrame({'cybase_struc_id':Struc_ids, 'struc_name':Struc_names})
        Struc_info['ID'] = ID
        return(Struc_info)
    else:
        return(pd.DataFrame([[np.NAN, np.NAN, ID]], columns=['cybase_struc_id', 'struc_name', 'ID']))

StructureInfo = pd.DataFrame()
for cbid in cbID:
    data_entry = BeautifulSoup(open("htmls/"+cbid+".html"), 'lxml')
    tables = data_entry.find_all('table', attrs={'class':'cardtable'})
    structure_info = get_structure_info(tables[3], cbid)
    StructureInfo = pd.concat([StructureInfo, structure_info], axis=0, ignore_index=True)
StructureInfo.head(10)

,cybase_struc_id,struc_name,ID
0,NaN,NaN,992
1,NaN,NaN,1610
2,NaN,NaN,913
3,NaN,NaN,25
4,NaN,NaN,1345
5,NaN,NaN,1574
6,NaN,NaN,1292
7,NaN,NaN,656
8,NaN,NaN,261
9,NaN,NaN,89


In [332]:
StructureInfo.to_excel('Cybase_Structure_Info.xlsx', header=True, index=False)

In [333]:
def ThreeD_structure_filter1(tag):
    return tag.name == 'fieldset' and tag.legend.text == '3D models'

def ThreeD_structure_filter2(tag):
    return tag.name == 'a' and tag.text == 'Download'

ThreeD_StructureInfo = pd.DataFrame()
for cbid in cbID:
    data_entry = BeautifulSoup(open("htmls/"+cbid+".html"), 'lxml')
    fieldset = data_entry.find(ThreeD_structure_filter1)
    if fieldset is not None:
        href = fieldset.find(ThreeD_structure_filter2)['href']
        ThreeD_id = re.compile(r'[0-9]+\.pdb').findall(href)[0]
        ThreeD_StructureInfo = pd.concat([ThreeD_StructureInfo, pd.DataFrame([[cbid, ThreeD_id]], columns=['ID','3D_id'])])
    else:
        ThreeD_StructureInfo = pd.concat([ThreeD_StructureInfo, pd.DataFrame([[cbid, np.NAN]], columns=['ID','3D_id'])])
ThreeD_StructureInfo.head(10)

,ID,3D_id
0,992,NaN
0,1610,NaN
0,913,NaN
0,25,25.pdb
0,1345,NaN
0,1574,NaN
0,1292,NaN
0,656,NaN
0,261,NaN
0,89,NaN


In [334]:
StructureInfo2 = pd.merge(StructureInfo, ThreeD_StructureInfo, how='outer', on='ID')
StructureInfo2 = StructureInfo2.reindex(columns=['ID', 'cybase_struc_id', 'struc_name', '3D_id'])
StructureInfo2.head(10)

,ID,cybase_struc_id,struc_name,3D_id
0,992,NaN,NaN,NaN
1,1610,NaN,NaN,NaN
2,913,NaN,NaN,NaN
3,25,NaN,NaN,25.pdb
4,1345,NaN,NaN,NaN
5,1574,NaN,NaN,NaN
6,1292,NaN,NaN,NaN
7,656,NaN,NaN,NaN
8,261,NaN,NaN,NaN
9,89,NaN,NaN,NaN


In [335]:
StructureInfo2.to_excel('Cybase_Structure_Info2.xlsx', header=True, index=False)

In [336]:
## get cybase structure list
url = 'https://www.cybase.org.au/index.php?table=structure&page=list&text=&textfield=All+fields&Class%5B%5D=ALL&Organism%5B%5D=ALL&Type%5B%5D=ALL&Assay%5B%5D=ALL&fields%5B%5D=ID&fields%5B%5D=Name&fields%5B%5D=Protein+name&fields%5B%5D=Class&fields%5B%5D=Type&fields%5B%5D=PDBcode'
struc_entrys = BeautifulSoup(open("Cybase_struc_list.html"), 'lxml')
struc_list = pd.read_html(struc_entrys.find('table', attrs={'id':'datatable'}).prettify())[0]
struc_list = struc_list.dropna(axis=0,how='all')
struc_list = struc_list.dropna(axis=1,how='all')
struc_list.columns = struc_list.iloc[0]
struc_list = struc_list.rename(columns={'ID':'cybase_struc_id'})
struc_list = struc_list[1:]

In [337]:
struc_list.head()

,cybase_struc_id,Name,Protein name,Class,Type,PDBcode
2,1,High-resolution solution structure of kalata B1,kalata B1,Cyclotide,NMR,1NB1
3,2,High-resolution solution structure of cyclovio...,cycloviolacin O1,Cyclotide,NMR,1NBJ
4,3,"Solution structure of MCoTI-II, a macrocyclic ...",MCoTI-II,Squash trypsin inhibitor,NMR,1IB9
5,4,Solution structure of the Moebius cyclotide ka...,kalata B2,Cyclotide,NMR,1PT4
6,5,Solution Structure of the Cylcotide Palicourei...,palicourein,Cyclotide,NMR,1R1F


In [338]:
StructureInfo3 = pd.merge(StructureInfo2, struc_list.loc[:,['cybase_struc_id', 'PDBcode']], 
                          how='left', on='cybase_struc_id')
StructureInfo3.head()

,ID,cybase_struc_id,struc_name,3D_id,PDBcode
0,992,NaN,NaN,NaN,NaN
1,1610,NaN,NaN,NaN,NaN
2,913,NaN,NaN,NaN,NaN
3,25,NaN,NaN,25.pdb,NaN
4,1345,NaN,NaN,NaN,NaN


In [339]:
StructureInfo3.to_excel('Cybase_Structure_Info3.xlsx', header=True, index=False)